In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
from scipy.optimize import minimize
TOLERANCE = 1e-35

In [2]:
import fix_yahoo_finance as yf
yf.pdr_override()

In [3]:
#You have to choose the assets
tickers = ['VT', 'BND', 'EEM', 'GSG']

num_assets = len(tickers)
pf_data = pd.DataFrame()

In [6]:
for t in tickers:
    pf_data[t] = pdr.get_data_yahoo(t, start = '2007-4-11', end = '2018-5-12')['Adj Close']

(pf_data/pf_data.iloc[0] * 100).plot(figsize = (10, 5))


[*********************100%***********************]  1 of 1 downloaded


KeyError: 'Adj Close'

In [7]:
#Statistics Des
log_returns = np.log(pf_data/pf_data.shift(1))
#Exp_ret = log_returns.mean()*252
Var_Cov_Mat = log_returns.cov()*252
print(Var_Cov_Mat)

Empty DataFrame
Columns: []
Index: []


In [8]:
#Portfolio weights
weights = np.multiply(np.ones(num_assets), 1/(num_assets))

Var_Port = np.dot(weights.T, np.dot(Var_Cov_Mat, weights))
print(Var_Port)

Vol_Port = Var_Port**0.5
print(Vol_Port)

ValueError: shapes (0,0) and (4,) not aligned: 0 (dim 1) != 4 (dim 0)

In [9]:
def _allocation_risk(weights, covariances):

    # We calculate the risk of the weights distribution
    portfolio_risk = (np.dot(weights.T, np.dot(covariances, weights)))**0.5

    # It returns the risk of the weights distribution
    return portfolio_risk

_allocation_risk(weights, Var_Cov_Mat)

def marginal_contribution(weights, Cov_i, portfolio_risk, i):
    MC = np.multiply(np.multiply(np.dot(weights.T, Cov_i), portfolio_risk**-1), weights[i])
    return MC


def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    # We calculate the risk of the weights distribution
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of the weights
    # distribution
    Var_Cov = np.matrix(covariances)
    
    assets_risk_contribution = np.ones(num_assets)
    for i in range(num_assets):
        x = np.array(Var_Cov[i]).T
        assets_risk_contribution[i] = marginal_contribution(weights, x, portfolio_risk, i)
    assets_risk_contribution /= sum(assets_risk_contribution)
    # It returns the contribution of each asset to the risk of the weights
    # distribution
    return assets_risk_contribution

_assets_risk_contribution_to_allocation_risk(weights, Var_Cov_Mat)


def _risk_budget_objective_error(weights, covariances):

    # We calculate the contribution of each asset to the risk of the weights
    # distribution
    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    # We calculate the desired contribution of each asset to the risk of the
    # weights distribution
    assets_risk_target = np.multiply(np.ones(num_assets), 1/(num_assets))
    
    # Error between the desired contribution and the calculated contribution of
    # each asset
    error = \
        sum(np.square(assets_risk_contribution - assets_risk_target))

    # It returns the calculated error
    return error

_risk_budget_objective_error(weights, Var_Cov_Mat)


def _get_risk_parity_weights(covariances, initial_weights):

    # Restrictions to consider in the optimisation: only long positions whose
    # sum equals 100%
        
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})

    # Optimisation process in scipy
    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=(covariances),
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    # Recover the weights from the optimised object
    weights = optimize_result.x

    # It returns the optimised weights
    return weights

weights_target = _get_risk_parity_weights(Var_Cov_Mat, weights)

# Convert the weights to a pandas Series
weights_target = pd.Series(weights_target, index = pf_data.columns, name='Weight')


ValueError: shapes (0,0) and (4,) not aligned: 0 (dim 1) != 4 (dim 0)